# Practical Exam: Spectrum Shades LLC
Spectrum Shades LLC is a prominent supplier of concrete color solutions, offering a wide range of pigments and coloring systems used in various concrete applications, including decorative concrete, precast concrete, and concrete pavers. The company prides itself on delivering high-quality colorants that meet the unique needs of its diverse clientele, including contractors, architects, and construction companies.
</br></br>
The company has recently observed a growing number of customer complaints regarding inconsistent color quality in their products. The discrepancies have led to a decline in customer satisfaction and a potential increase in product returns.
By identifying and mitigating the factors causing color variations, the company can enhance product reliability, reduce customer complaints, and minimize return rates.
</br></br>
You are part of the data analysis team tasked with providing actionable insights to help Spectrum Shades LLC address the issues of inconsistent color quality and improve customer satisfaction.

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `production_data` table, and the data team have provided the following data description. 

Write a query to ensure the data matches the description provided, including identifying and cleaning all invalid values. You must match all column names and description criteria.
</br>

- You should start with the data in the file "production_data.csv".
- Your output should be a DataFrame named clean_data.
- All column names and values should match the table below.
</br>

| Column Name             | Criteria                                                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|
| batch_id | Discrete. Identifier for each batch. Missing values are not possible. |
| production_date | Date. Date when the batch was produced.|
| raw_material_supplier | Categorical. Supplier of the raw materials. (1='national_supplier', 2='international_supplier'). <br> Missing values should be replaced with 'national_supplier'.|
| pigment_type           | Nominal. Type of pigment used. ['type_a', 'type_b', 'type_c']. <br> Missing values should be replaced with 'other'. |
| pigment_quantity       | Continuous. Amount of pigment added (in kilograms) (Range: 1 - 100). <br> Missing values should be replaced with median. |
| mixing_time           | Continuous. Duration of the mixing process (in minutes). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |
| mixing_speed          | Categorical. Speed of the mixing process represented as categories: 'Low', 'Medium', 'High'.</br> Missing values should be replaced with 'Not Specified'. |
| product_quality_score | Continuous. Overall quality score of the final product (rating on a scale of 1 to 10). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |


In [ ]:
import pandas as pd
import numpy as np

production_df= pd.read_csv("production_data.csv")

# pd_df.head()

clean_data= production_df.copy()

# production_date part
clean_data['production_date'] = pd.to_datetime(clean_data['production_date'], errors='coerce')

# raw_material_supplier part
clean_data['raw_material_supplier'] = clean_data['raw_material_supplier'].replace({
    1: 'national_supplier',
    2: 'international_supplier'
})

clean_data['raw_material_supplier'] = clean_data['raw_material_supplier'].astype(str).str.strip().str.lower()

clean_data['raw_material_supplier'] = clean_data['raw_material_supplier'].replace({
    '1': 'national_supplier',
    '2': 'international_supplier'
})

valid_suppliers = ['national_supplier', 'international_supplier']
clean_data['raw_material_supplier'] = clean_data['raw_material_supplier'].apply(
    lambda x: x if x in valid_suppliers else 'national_supplier'
)


# pigment_type part
valid_pigment_types = ['type_a', 'type_b', 'type_c']

clean_data['pigment_type'] = clean_data['pigment_type'].astype(str).str.strip().str.lower()
clean_data['pigment_type'] = clean_data['pigment_type'].apply(
    lambda x: x if x in valid_pigment_types else 'other'
)

# pigment_quantity part
clean_data.loc[(clean_data['pigment_quantity'] < 1) | (clean_data['pigment_quantity'] > 100), 'pigment_quantity'] = np.nan

median_pigment_quantity = clean_data['pigment_quantity'].median()
clean_data['pigment_quantity'] = clean_data['pigment_quantity'].fillna(median_pigment_quantity)

# mixing_time part
mean_mixing_time = clean_data['mixing_time'].mean()
clean_data['mixing_time'] = clean_data['mixing_time'].fillna(mean_mixing_time).round(2)

# mixing_speed part
valid_mixing_speeds = ['Low', 'Medium', 'High']
clean_data['mixing_speed'] = clean_data['mixing_speed'].astype(str).str.strip().str.title()
clean_data['mixing_speed'] = clean_data['mixing_speed'].apply(
    lambda x: x if x in valid_mixing_speeds else 'Not Specified'
)

#product_quality_score
clean_data.loc[(clean_data['product_quality_score'] < 1) | (clean_data['product_quality_score'] > 10), 'product_quality_score'] = np.nan

mean_product_quality_score = clean_data['product_quality_score'].mean()
clean_data['product_quality_score'] = clean_data['product_quality_score'].fillna(mean_product_quality_score).round(2)

# visualizing output
clean_data.info()

clean_data.head()

clean_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   batch_id               2000 non-null   int64         
 1   production_date        2000 non-null   datetime64[ns]
 2   raw_material_supplier  2000 non-null   object        
 3   pigment_type           2000 non-null   object        
 4   pigment_quantity       2000 non-null   float64       
 5   mixing_time            2000 non-null   float64       
 6   mixing_speed           2000 non-null   object        
 7   product_quality_score  2000 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 125.1+ KB


batch_id                 0
production_date          0
raw_material_supplier    0
pigment_type             0
pigment_quantity         0
mixing_time              0
mixing_speed             0
product_quality_score    0
dtype: int64

In [44]:
#clean_data.head()

,batch_id,production_date,raw_material_supplier,pigment_type,pigment_quantity,mixing_time,mixing_speed,product_quality_score
0,1,2024-06-25,national_supplier,type_a,42.822881,22.56,High,7.17
1,2,2023-11-23,international_supplier,type_b,42.873479,44.97,High,6.85
2,3,2024-02-18,international_supplier,type_b,33.251206,52.62,High,5.66
3,4,2023-11-11,national_supplier,type_a,52.532817,24.96,High,6.99
4,5,2024-04-11,international_supplier,type_c,36.205108,25.37,High,7.10


# Task 2

You want to understand how the supplier type and quantity of materials affect the final product attributes.

Calculate the average `product_quality_score` and `pigment_quantity` grouped by `raw_material_supplier`.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named aggregated_data.
- It should include the three columns: `raw_material_supplier`, `avg_product_quality_score`, and `avg_pigment_quantity`.
- Your answers should be rounded to 2 decimal places.


In [ ]:

aggregated_data = clean_data.groupby('raw_material_supplier').agg(
    avg_product_quality_score=('product_quality_score', 'mean'),
    avg_pigment_quantity=('pigment_quantity', 'mean')
).round(2)

aggregated_data = aggregated_data.reset_index()

print(aggregated_data)



    raw_material_supplier  avg_product_quality_score  avg_pigment_quantity
0  international_supplier                       5.97                 34.91
1       national_supplier                       8.02                 44.73


# Task 3

To get more insight into the factors behind product quality, you want to filter the data to see an average product quality score for a specified set of results.

Identify the average `product_quality_score` for batches with a `raw_material_supplier` of 2 and a `pigment_quantity` greater than 35 kg.

Write a query to return the average `avg_product_quality_score` for these filtered batches. Use the original production data table, not the output of Task 2.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named pigment_data.
- It should consist of a 1-row DataFrame with 3 columns: `raw_material_supplier`, `pigment_quantity`, and `avg_product_quality_score`.
- Your answers should be rounded to 2 decimal places where appropriate.


In [ ]:

production_df= pd.read_csv("production_data.csv")

filtered = production_df[
    (production_df['raw_material_supplier'] ==2) &
    (production_df['pigment_quantity'] >35)
]

avg_quality_score= filtered['product_quality_score'].mean()

pigment_data= pd.DataFrame({
    'raw_material_supplier': [2],
    'pigment_quantity': ['>35 kg'],
    'avg_product_quality_score': [round(avg_quality_score,2)]
})

print(pigment_data)

   raw_material_supplier pigment_quantity  avg_product_quality_score
0                      2           >35 kg                       5.97


# Task 4

In order to proceed with further analysis later, you need to analyze how various factors relate to product quality. Start by calculating the mean and standard deviation for the following columns: `pigment_quantity`, and `product_quality_score`. </br> These statistics will help in understanding the central tendency and variability of the data related to product quality.
</br> </br >
Next, calculate the Pearson correlation coefficient between the following variables: `pigment_quantity`, and `product_quality_score`.
</br>
These correlation coefficients will provide insights into the strength and direction of the relationships between the factors and overall product quality.


- You should start with the data in the file 'production_data.csv'.
- Calculate the mean and standard deviation for the columns pigment_quantity and product_quality_score as: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`.
- Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score as: `corr_coef`
- Your output should be a DataFrame named product_quality.
- It should include the columns: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`, `corr_coef`.
- Ensure that your answers are rounded to 2 decimal places.


In [ ]:

product_quality_score_mean = production_df['product_quality_score'].mean()
product_quality_score_sd = production_df['product_quality_score'].std()
pigment_quantity_mean = production_df['pigment_quantity'].mean()
pigment_quantity_sd = production_df['pigment_quantity'].std()

corr_coef = production_df['pigment_quantity'].corr(production_df['product_quality_score'])

product_quality = pd.DataFrame({
    'product_quality_score_mean': [round(product_quality_score_mean, 2)],
    'product_quality_score_sd': [round(product_quality_score_sd, 2)],
    'pigment_quantity_mean': [round(pigment_quantity_mean, 2)],
    'pigment_quantity_sd': [round(pigment_quantity_sd, 2)],
    'corr_coef': [round(corr_coef, 2)]
})

print(product_quality)

print(product_quality.columns)

   product_quality_score_mean  ...  corr_coef
0                        6.68  ...       0.49

[1 rows x 5 columns]
Index(['product_quality_score_mean', 'product_quality_score_sd',
       'pigment_quantity_mean', 'pigment_quantity_sd', 'corr_coef'],
      dtype='object')


# FORMATTING AND NAMING CHECK
Use the code block below to check that your outputs are correctly named and formatted before you submit your project.

This code checks whether you have met our automarking requirements: that the specified DataFrames exist and contain the required columns. It then prints a table showing ✅ for each column that exists and ❌ for any that are missing, or if the DataFrame itself isn't available.

If a DataFrame or a column in a DataFrame doesn't exist, carefully check your code again.

IMPORTANT: even if your code passes the check below, this does not mean that your entire submission is correct. This is a check for naming and formatting only.

In [32]:
import pandas as pd

def check_columns(output_df, output_df_name, required_columns):
    results = []
    for col in required_columns:
        exists = col in output_df.columns
        results.append({'Dataset': output_df_name, 'Column': col, 'Exists': '✅' if exists else '❌'})
    return results

def safe_check(output_df_name, required_columns):
    results = []
    if output_df_name in globals():
        obj = globals()[output_df_name]
        if isinstance(obj, pd.DataFrame):
            results.extend(check_columns(obj, output_df_name, required_columns))
        elif isinstance(obj, str) and ("SELECT" in obj.upper() or "FROM" in obj.upper()):
            results.append({'Dataset': output_df_name, 'Column': '—', 'Exists': 'ℹ️ SQL query string'})
        else:
            results.append({'Dataset': output_df_name, 'Column': '—', 'Exists': '❌ Not a DataFrame or query'})
    else:
        results.append({'Dataset': output_df_name, 'Column': '—', 'Exists': '❌ Variable not defined'})
    return results

requirements = {
    'clean_data': ['production_date', 'pigment_type', 'mixing_time', 'mixing_speed'],
    'aggregated_data': ['raw_material_supplier', 'avg_product_quality_score', 'avg_pigment_quantity'],
    'pigment_data': ['raw_material_supplier', 'pigment_quantity', 'avg_product_quality_score'],
    'product_quality': ['product_quality_score_mean', 'product_quality_score_sd',
                        'pigment_quantity_mean', 'pigment_quantity_sd', 'corr_coef']
}

all_results = []
for output_df_name, cols in requirements.items():
    all_results += safe_check(output_df_name, cols)

check_results_df = pd.DataFrame(all_results)

print(check_results_df)

            Dataset                      Column Exists
0        clean_data             production_date      ✅
1        clean_data                pigment_type      ✅
2        clean_data                 mixing_time      ✅
3        clean_data                mixing_speed      ✅
4   aggregated_data       raw_material_supplier      ✅
5   aggregated_data   avg_product_quality_score      ✅
6   aggregated_data        avg_pigment_quantity      ✅
7      pigment_data       raw_material_supplier      ✅
8      pigment_data            pigment_quantity      ✅
9      pigment_data   avg_product_quality_score      ✅
10  product_quality  product_quality_score_mean      ✅
11  product_quality    product_quality_score_sd      ✅
12  product_quality       pigment_quantity_mean      ✅
13  product_quality         pigment_quantity_sd      ✅
14  product_quality                   corr_coef      ✅
